# CPSC 330 - Applied Machine Learning 

## Homework 8: Time series
**Due date: [April 10, 11:59pm](https://github.com/UBC-CS/cpsc330-2023W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [184]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W2/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset. We will be forcasting average avocado price for the next week. 

In [185]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [186]:
df.shape

(18249, 13)

In [187]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [188]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [189]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [190]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 

_Points:_ 4

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture, we had different measurements for each Location. How about this avocado sales dataset? For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [191]:
df_train.groupby("Date").count().reset_index().sort_values("region", ascending=True).set_index("Date")

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
Date,,,,,,,,,,,,
2017-06-18,107,107,107,107,107,107,107,107,107,107,107,107
2017-06-25,107,107,107,107,107,107,107,107,107,107,107,107
2015-12-06,107,107,107,107,107,107,107,107,107,107,107,107
2015-01-04,108,108,108,108,108,108,108,108,108,108,108,108
2016-10-09,108,108,108,108,108,108,108,108,108,108,108,108
...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-13,108,108,108,108,108,108,108,108,108,108,108,108
2015-12-20,108,108,108,108,108,108,108,108,108,108,108,108
2015-12-27,108,108,108,108,108,108,108,108,108,108,108,108


In [192]:
df_train.groupby("region").count()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year
region,,,,,,,,,,,,
Albany,286,286,286,286,286,286,286,286,286,286,286,286
Atlanta,286,286,286,286,286,286,286,286,286,286,286,286
BaltimoreWashington,286,286,286,286,286,286,286,286,286,286,286,286
Boise,286,286,286,286,286,286,286,286,286,286,286,286
Boston,286,286,286,286,286,286,286,286,286,286,286,286
BuffaloRochester,286,286,286,286,286,286,286,286,286,286,286,286
California,286,286,286,286,286,286,286,286,286,286,286,286
Charlotte,286,286,286,286,286,286,286,286,286,286,286,286
Chicago,286,286,286,286,286,286,286,286,286,286,286,286


In [193]:
df_train.groupby(["type", "region"]).count()

Date  AveragePrice  Total Volume  4046  \
type         region                                                        
conventional Albany                143           143           143   143   
             Atlanta               143           143           143   143   
             BaltimoreWashington   143           143           143   143   
             Boise                 143           143           143   143   
             Boston                143           143           143   143   
...                                ...           ...           ...   ...   
organic      Syracuse              143           143           143   143   
             Tampa                 143           143           143   143   
             TotalUS               143           143           143   143   
             West                  143           143           143   143   
             WestTexNewMexico      140           140           140   140   

                                  4225  4770  Total Bags  Small Bags  \
type         region                                                    
conventional Albany                143   143         143         143   
             Atlanta               143   143         143         143   
             BaltimoreWashington   143   143         143         143   
             Boise                 143   143         143         143   
             Boston                143   143         143         143   
...                                ...   ...         ...         ...   
organic      Syracuse              143   143         143         143   
             Tampa                 143   143         143         143   
             TotalUS               143   143         143         143   
             West                  143   143         143         143   
             WestTexNewMexico      140   140         140         140   

                                  Large Bags  XLarge Bags  year  
type         region                                              
conventional Albany                      143          143   143  
             Atlanta                     143          143   143  
             BaltimoreWashington         143          143   143  
             Boise                       143          143   143  
             Boston                      143          143   143  
...                                      ...          ...   ...  
organic      Syracuse                    143          143   143  
             Tampa                       143          143   143  
             TotalUS                     143          143   143  
             West                        143          143   143  
             WestTexNewMexico            140          140   140  

[108 rows x 11 columns]

There are different time series for each `region` and `type` combination. Additionally, each of `2017-06-18`, `2017-06-25`, and `2015-12-06` are missing from `type`=organic and `region`=WestTexNewMexico.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 

_Points:_ 4

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [194]:
first_day = df_train["Date"].min()

df_train = df_train.assign(WeeksSince=df_train["Date"].apply(lambda x: ((x - first_day).days / 7)))
df_train

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,WeeksSince
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany,51.0
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany,50.0
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany,49.0
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany,48.0
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,2017-01-29,1.30,17839.37,1486.34,4498.48,26.12,11828.43,11821.76,6.67,0.0,organic,2017,WestTexNewMexico,108.0
47,2017-01-22,1.21,16430.64,1413.93,2820.53,20.25,12175.93,12073.07,102.86,0.0,organic,2017,WestTexNewMexico,107.0
48,2017-01-15,1.19,17014.23,1203.87,2904.22,23.07,12883.07,12476.57,406.50,0.0,organic,2017,WestTexNewMexico,106.0
49,2017-01-08,1.18,14375.39,1327.98,2617.20,5.75,10424.46,10283.85,140.61,0.0,organic,2017,WestTexNewMexico,105.0


In [195]:
df_train["WeeksSince"].unique()

array([ 51.,  50.,  49.,  48.,  47.,  46.,  45.,  44.,  43.,  42.,  41.,
        40.,  39.,  38.,  37.,  36.,  35.,  34.,  33.,  32.,  31.,  30.,
        29.,  28.,  27.,  26.,  25.,  24.,  23.,  22.,  21.,  20.,  19.,
        18.,  17.,  16.,  15.,  14.,  13.,  12.,  11.,  10.,   9.,   8.,
         7.,   6.,   5.,   4.,   3.,   2.,   1.,   0., 103., 102., 101.,
       100.,  99.,  98.,  97.,  96.,  95.,  94.,  93.,  92.,  91.,  90.,
        89.,  88.,  87.,  86.,  85.,  84.,  83.,  82.,  81.,  80.,  79.,
        78.,  77.,  76.,  75.,  74.,  73.,  72.,  71.,  70.,  69.,  68.,
        67.,  66.,  65.,  64.,  63.,  62.,  61.,  60.,  59.,  58.,  57.,
        56.,  55.,  54.,  53.,  52., 142., 141., 140., 139., 138., 137.,
       136., 135., 134., 133., 132., 131., 130., 129., 128., 127., 126.,
       125., 124., 123., 122., 121., 120., 119., 118., 117., 116., 115.,
       114., 113., 112., 111., 110., 109., 108., 107., 106., 105., 104.])

It appears that all the time series records are one week apart except for the previously mentioned region-type group that is likely missing 3 intervals somewhere in between.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 

_Points:_ 4

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [196]:
df_train["region"].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

There is some overlap in the regions. Some regions are cities (eg. SanFrancisco), some regions are states (eg. California), and some are larger (eg. TotalUS). The examples provided all include SanFrancisco.

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2023W2/blob/main/lectures/mathias/19_time-series.ipynb), with some improvements.

In [197]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [198]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [199]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [200]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

### 1.4 `AveragePrice` baseline 

_Points:_ 4

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Type your answer here, replacing this text._

In [201]:
train_r2 = r2_score(df_train["AveragePriceNextWeek"], df_train["AveragePrice"])
train_r2

0.8285800937261841

In [202]:
test_r2 = r2_score(df_test["AveragePriceNextWeek"], df_test["AveragePrice"])
test_r2

0.7631780188583048

We get an $R^2$ value of 0.829 for the train set and an $R^2$ value of 0.763 for the test set.

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price

_Points:_ 10

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. We got to 0.80, but not beyond that. Let us know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

Code references [lecture 19](https://github.com/UBC-CS/cpsc330-2023W2/blob/5a6fd0c9be17b08752a5e8d3f003168fae0569ea/lectures/mehrdad/19_time-series.ipynb)

In [203]:
df_train.columns

Index(['Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770',
       'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year',
       'region', 'AveragePriceNextWeek'],
      dtype='object')

In [204]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15441 entries, 0 to 18222
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  15441 non-null  datetime64[ns]
 1   AveragePrice          15441 non-null  float64       
 2   Total Volume          15441 non-null  float64       
 3   4046                  15441 non-null  float64       
 4   4225                  15441 non-null  float64       
 5   4770                  15441 non-null  float64       
 6   Total Bags            15441 non-null  float64       
 7   Small Bags            15441 non-null  float64       
 8   Large Bags            15441 non-null  float64       
 9   XLarge Bags           15441 non-null  float64       
 10  type                  15441 non-null  object        
 11  year                  15441 non-null  int64         
 12  region                15441 non-null  object        
 13  AveragePriceNextWeek 

In [205]:
numeric_features = [
  "AveragePrice",
  "Total Volume",
  "4046",
  "4225",
  "4770",
  "Total Bags",
  "Small Bags",
  "Large Bags",
  "XLarge Bags",
]
categorical_features = [
  "type",
  "year",
  "region",
]
drop_features = ["Date"]
target = ["AveragePriceNextWeek"]

In [206]:
def preprocess_features(
  train_df,
  test_df,
  numeric_features,
  categorical_features,
  drop_features,
  target
):

  all_features = set(numeric_features + categorical_features + drop_features + target)
  if set(train_df.columns) != all_features:
    print("Missing columns", set(train_df.columns) - all_features)
    print("Extra columns", all_features - set(train_df.columns))
    raise Exception("Columns do not match")

  numeric_transformer = make_pipeline(
    SimpleImputer(strategy="median"), StandardScaler()
  )
  categorical_transformer = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="missing"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False),
  )

  preprocessor = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features),
    ("drop", drop_features),
  )
  preprocessor.fit(train_df)
  ohe_feature_names = (
    preprocessor.named_transformers_["pipeline-2"]
    .named_steps["onehotencoder"]
    .get_feature_names_out(categorical_features)
    .tolist()
  )
  new_columns = numeric_features + ohe_feature_names

  X_train_enc = pd.DataFrame(
    preprocessor.transform(train_df), index=train_df.index, columns=new_columns
  )
  X_test_enc = pd.DataFrame(
    preprocessor.transform(test_df), index=test_df.index, columns=new_columns
  )

  y_train = train_df["AveragePriceNextWeek"]
  y_test = test_df["AveragePriceNextWeek"]

  return X_train_enc, y_train, X_test_enc, y_test, preprocessor

Don't Encode Anything

In [207]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features,
    categorical_features,
    drop_features,
    target
)

In [208]:
X_train_enc.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type_conventional,...,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
0,-0.432512,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.383676,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.554604,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.823205,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.994133,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [209]:
lr = Ridge()
lr.fit(X_train_enc, y_train)
lr.score(X_train_enc, y_train)

0.8459817408490182

In [210]:
lr.score(X_test_enc, y_test)

0.7953017368914064

Encode Month

In [211]:
df_train_month = df_train.assign(month=df_train["Date"].apply(lambda x: x.month_name()))
df_test_month = df_test.assign(month=df_test["Date"].apply(lambda x: x.month_name()))
df_train_month.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,month
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,January
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17,January
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06,January
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99,January
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99,February


In [212]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_month,
    df_test_month,
    numeric_features,
    categorical_features + ["month"],
    drop_features,
    target
)

In [213]:
X_train_enc.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type_conventional,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,-0.432512,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.383676,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.554604,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.823205,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.994133,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
lr = Ridge()
lr.fit(X_train_enc, y_train)
lr.score(X_train_enc, y_train)

0.850408544826414

In [215]:
lr.score(X_test_enc, y_test)

0.8018263108038561

Encode Season

In [216]:
def get_season(month):
  WINTER_MONTHS = ["December", "January", "February"]
  AUTUMN_MONTHS = ["September", "October", "November"]
  SUMMER_MONTHS = ["June", "July", "August"] 
  SPRING_MONTHS = ["March", "April", "May"]
  if month in WINTER_MONTHS:
    return "Winter"
  elif month in AUTUMN_MONTHS:
    return "Autumn"
  elif month in SUMMER_MONTHS:
    return "Summer"
  else:
    return "Spring"

In [217]:
df_train_season = df_train.assign(season=df_train["Date"].apply(lambda x: get_season(x.month_name())))
df_test_season = df_test.assign(season=df_test["Date"].apply(lambda x: get_season(x.month_name())))
df_train_season.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,season
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,Winter
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17,Winter
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06,Winter
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99,Winter
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99,Winter


In [218]:
df_train_season["season"].value_counts()

season
Spring    4320
Summer    4210
Winter    3671
Autumn    3240
Name: count, dtype: int64

In [219]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_season,
    df_test_season,
    numeric_features,
    categorical_features + ["season"],
    drop_features,
    target
)

In [220]:
X_train_enc.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type_conventional,...,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico,season_Autumn,season_Spring,season_Summer,season_Winter
0,-0.432512,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.383676,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.554604,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.823205,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.994133,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [221]:
lr = Ridge()
lr.fit(X_train_enc, y_train)
lr.score(X_train_enc, y_train)

0.8486740844200468

In [222]:
lr.score(X_test_enc, y_test)

0.7937165884853135

Encode Month and Season

In [223]:
df_train_month_season = df_train_month.assign(season=df_train_month["month"].apply(get_season))
df_test_month_season = df_test_month.assign(season=df_test_month["month"].apply(get_season))
df_train_month_season.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,month,season
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,January,Winter
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17,January,Winter
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06,January,Winter
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99,January,Winter
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99,February,Winter


In [224]:
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_month_season,
    df_test_month_season,
    numeric_features,
    categorical_features + ["month", "season"],
    drop_features,
    target
)

In [225]:
X_train_enc.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type_conventional,...,month_June,month_March,month_May,month_November,month_October,month_September,season_Autumn,season_Spring,season_Summer,season_Winter
0,-0.432512,-0.234535,-0.229503,-0.222203,-0.214954,-0.232206,-0.229907,-0.223154,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.383676,-0.234440,-0.230948,-0.219448,-0.214272,-0.233587,-0.231513,-0.223789,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-0.554604,-0.233469,-0.231018,-0.219530,-0.214196,-0.229850,-0.226469,-0.224325,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.823205,-0.233283,-0.230996,-0.218170,-0.213945,-0.230999,-0.228629,-0.222193,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.994133,-0.225747,-0.230668,-0.196131,-0.213811,-0.232627,-0.229930,-0.224856,-0.172063,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [226]:
lr = Ridge()
lr.fit(X_train_enc, y_train)
lr.score(X_train_enc, y_train)

0.8504085515939936

In [227]:
lr.score(X_test_enc, y_test)

0.8018230342072037

Try encoding month and day, and ignoring the year column. 

In [228]:
# Month and Day feature
def create_month_day_feature(features, separate=False):
    new_df = features.copy()
    if separate:
        month_day_array = np.array([(date.month_name(), date.day) for date in features["Date"]], dtype=object)
        new_df["month"] = month_day_array[:, 0]
        new_df["day"] = month_day_array[:, 1]
    else:
        month_day_array = np.array([(date.month_name() + " " + str(date.day)) for date in features["Date"]], dtype=object)
        new_df["month_day"] = month_day_array
    return new_df

In [229]:
separate_columns = False # setting this to true gives a lower training accuracy
df_train_month_day = create_month_day_feature(df_train, separate_columns)
df_test_month_day = create_month_day_feature(df_test, separate_columns)

df_train_month_day.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek,month_day
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24,January 4
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17,January 11
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06,January 18
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99,January 25
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99,February 1


In [230]:
month_day_column_names = ["month", "day"] if separate_columns else ["month_day"]
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train_month_day,
    df_test_month_day,
    numeric_features,
    ["type", "region"] + month_day_column_names,
    drop_features + ["year"],
    target
)

In [231]:
lr = Ridge()
lr.fit(X_train_enc, y_train)
lr.score(X_train_enc, y_train)

0.863190135475175

In [232]:
lr.score(X_test_enc, y_test)

0.8044312330640084

We first tried encoding month, season, and both together with Ridge. We ignored day of week since it's the same all the time, and year already exists. Month encoding had the best results at 0.850 and 0.802 for train and test respectively. Season was worse at 0.849 and 0.794 for train and test respectively. Encoding both was just around the same as Month. No encoding netted a 0.846 and 0.795 for train and test respectively. Overall, no encoding got us above the baseline of 0.79, but adding month encoding improved it to the best among all the attempts. Further adding season encoding might be unnecessary since it barely changes the month encoding score.

We also tried encoding month and day of the month as a single categorical feature (e.g. January 11, March 24, etc.), while also dropping the year column. The idea was to capture more granular seasonal patterns throughout the year. This resulted in a training accuracy of 0.863, and a test accuracy of 0.804, the highest score seen in all attempts. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: very short answer questions

Each question is worth 2 points.

<!-- BEGIN QUESTION -->

### 2.1 Time series

_Points:_ 4

The following questions pertain to Lecture 19 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.

<div class="alert alert-warning">

Solution_2.1
    
</div>

1. Cases where time points are unequally spaced are time series that track incidents such as earthquakes, 911 calls, hospital visits, and more. In these cases it's possible for multiple records of earthquakes, for example, to take place within an hour or none within a year.

2. Both approaches will struggle. For example, if we extract month information from the time point and records are skewed to take place in one month, then there will be a lack of sufficient data to train the model properly. However, creating lagged features will most likely struggle more since the lag doesn't always take into consideration the time since the last record. One earthquake incident record might also show the magnitude of the last earthquake which occurred 10 minutes ago while another might show the magnitude of the last earthquake which occurred 1 year ago. The one year ago lag might be completely irrelevant by then which would confuse the model.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Survival analysis

_Points:_ 6

The following questions pertain to Lecture 20 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer?
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.2
    
</div>

1. By labeling customers as only "churned" or "not churned" and using standard supervised learning techniques, we ignore certain aspects relevant in survival analysis like the time until churn occurs or when the customer started their tenure. There is usually some kind of censoring that limits the information that can be observed from the data.

2. Based on the survival functions plotted in the lecture notes, it would probably be customer B who will leave the service first. However, there are a lot of contextual information that need to be considered when deciding. For example, customer A might be using a one-month free trial and not be willing to extend beyond that. They both might also be of different demographics or have committed to the service by different lengths. Join time isn't usually enough.

3. A flat survival function indicates that the probability of the customer leaving is stable rather than declining over time. This could mean that the service provider is properly employing retention strategies to keep them satisfied.

<!-- END QUESTION -->

<br><br>

**PLEASE READ BEFORE YOU SUBMIT:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using [PrairieLearn](https://ca.prairielearn.com/pl/course_instance/6697). Don't forget to rename your file `hw8_sol.ipynb`.

![](img/eva-well-done.png)